Temporary dump location to resolve the issue of me forgetting to push some code and writing across two devices

To do:
 - ODEint warning is from a Stiff Equation. "hole" or jump in our solution points that occurs from having too wide timesteps
May want to look into other integration methods to resolve stiffness https://docs.scipy.org/doc/scipy-1.7.1/reference/generated/scipy.integrate.solve_ivp.html#scipy.integrate.solve_ivp

Moving to irregular grid
For each initial condition (and from REAL SOLUTION)
 - arbitrary # of time points (big enough)
 - arbitrary spacing between them
 - sample section @ those time points
 - fit a polynomial (cubic spline)
 - sample from the fit on a regular grid
 - Run SINDy on regular grid fit
 - test the solution MSE (SINDy - real)
 - MSE (polynomial values - real) as an extra
 
 

In [ ]:
# code snippet, run w/ len(tvals) 500 and 5000 to see difference
# will need supporting code tho
test
htest = 1 / (1+test**5)
#plt.scatter(test, htest)
feature_count =2
netswitch_x0s = (np.random.rand(n_of_ic, feature_count) * 5 - np.random.rand(n_of_ic, feature_count))
model = ps.SINDy(feature_names=['u', 'v'], optimizer=ps.STLSQ(threshold=0.7), feature_library=netswitch_library)
model.fit(list(netswitch_train_multi), t=dt, multiple_trajectories=True)
model.print()
netswitch_sim = model.simulate(netswitch_x0s[0], t=tvals, integrator="odeint")
test2 = netswitch_sim[:, 0]
htest2 = 1 / (1+test2**5)
plt.scatter(tvals, test2)